In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import os
from PIL import Image

from mnistm import MNISTMDataset

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 4080'

# Data Preprocessing

In [3]:
# Define transformations
transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to the same size as used in SVHN)
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel RGB
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize (assuming grayscale, same value for all channels)
])

transform_mnist_m = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB
])

transform_svhn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_usps = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to the same size as used in SVHN)
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel RGB
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize (assuming grayscale, same value for all channels)
])

In [4]:
# Load datasets
mnist_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
#mnist_m_dataset = datasets.ImageFolder(root='./mnist_m', transform=transform_mnist_m)
mnistm_train_dataset = MNISTMDataset("./datasets", train = True, transform=transform_mnist_m)
mnistm_test_dataset = MNISTMDataset("./datasets", train = False, transform=transform_mnist_m)

svhn_train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
svhn_test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)

usps_train_dataset = datasets.USPS(root='./data', train=True, transform=transform_usps, download=True)
usps_test_dataset = datasets.USPS(root='./data', train=False, transform=transform_usps, download=True)

Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat


In [7]:
batch_size = 256  # Set the batch size

total_size = len(mnist_train_dataset)
val_size = int(0.15 * total_size)
train_size = total_size - val_size
train_dataset, val_dataset = random_split(mnist_train_dataset, [train_size, val_size]) # Split the dataset

mnist_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
mnist_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_test_dataset, batch_size=batch_size, shuffle=False)

total_size = len(mnistm_train_dataset)
val_size = int(0.15 * total_size)
train_size = total_size - val_size
train_dataset, val_dataset = random_split(mnistm_train_dataset, [train_size, val_size]) # Split the dataset

mnistm_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
mnistm_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
mnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=batch_size, shuffle=False)

total_size = len(svhn_train_dataset)
val_size = int(0.15 * total_size)
train_size = total_size - val_size
train_dataset, val_dataset = random_split(svhn_train_dataset, [train_size, val_size]) # Split the dataset
svhn_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
svhn_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
svhn_test_loader = DataLoader(svhn_test_dataset, batch_size=batch_size, shuffle=False)

usps_train_loader = DataLoader(usps_train_dataset, batch_size=batch_size, shuffle=True)
usps_test_loader = DataLoader(usps_test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
'''
combined_dataset = ConcatDataset([mnist_train_dataset, mnistm_train_dataset])

total_size = len(combined_dataset)

val_size = int(0.15 * total_size) # Define the size of validation set (15% of the total dataset)
train_size = total_size - val_size

train_dataset, val_dataset = random_split(combined_dataset, [train_size, val_size]) # Split the dataset

test_dataset = ConcatDataset([mnist_test_dataset, mnistm_test_dataset])

batch_size = 64  # Set the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

mnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=64, shuffle=False)
svhn_test_loader = DataLoader(svhn_test_dataset, batch_size=64, shuffle=False)
'''

'\ncombined_dataset = ConcatDataset([mnist_train_dataset, mnistm_train_dataset])\n\ntotal_size = len(combined_dataset)\n\nval_size = int(0.15 * total_size) # Define the size of validation set (15% of the total dataset)\ntrain_size = total_size - val_size\n\ntrain_dataset, val_dataset = random_split(combined_dataset, [train_size, val_size]) # Split the dataset\n\ntest_dataset = ConcatDataset([mnist_test_dataset, mnistm_test_dataset])\n\nbatch_size = 64  # Set the batch size\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\nval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)\ntest_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)\n\nmnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=64, shuffle=False)\nsvhn_test_loader = DataLoader(svhn_test_dataset, batch_size=64, shuffle=False)\n'

# Model Architecture

In [6]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Adjust the input channels because input images are RGB images
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)  # Adjust for the size if you're not resizing to 32x32
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Model Training

In [18]:
def train_model(model, train_loader, val_loader, test_loader, num_epochs=10):
    # Send the model to the device (GPU or CPU)
    model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        print("For epoch: ", epoch)
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss, val_accuracy = evaluate_model(model, val_loader, device, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Test loop
    model.eval()
    test_loss, test_accuracy = evaluate_model(model, test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    test_loss, test_accuracy = evaluate_model(model, svhn_test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy for SVHN data: {test_accuracy:.4f}")
    test_loss, test_accuracy = evaluate_model(model, usps_test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy for USPS data: {test_accuracy:.4f}")
    test_loss, test_accuracy = evaluate_model(model, mnist_test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy for MNIST data: {test_accuracy:.4f}")
    
def evaluate_model(model, loader, device, criterion):
    total = 0
    correct = 0
    running_loss = 0.0
    with torch.no_grad():  # No gradients needed for validation/testing
        for data in loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = running_loss / len(loader)
    accuracy = correct / total
    return loss, accuracy

In [19]:
model = LeNet5()
train_model(model, mnistm_train_loader, mnistm_val_loader, mnistm_test_loader, num_epochs = 10)

For epoch:  0
Epoch 1/10, Train Loss: 1.1746, Train Accuracy: 0.6123
Epoch 1/10, Validation Loss: 0.6060, Validation Accuracy: 0.8146
For epoch:  1
Epoch 2/10, Train Loss: 0.4405, Train Accuracy: 0.8655
Epoch 2/10, Validation Loss: 0.3726, Validation Accuracy: 0.8826
For epoch:  2
Epoch 3/10, Train Loss: 0.3058, Train Accuracy: 0.9064
Epoch 3/10, Validation Loss: 0.2917, Validation Accuracy: 0.9120
For epoch:  3
Epoch 4/10, Train Loss: 0.2523, Train Accuracy: 0.9238
Epoch 4/10, Validation Loss: 0.2541, Validation Accuracy: 0.9247
For epoch:  4
Epoch 5/10, Train Loss: 0.2208, Train Accuracy: 0.9331
Epoch 5/10, Validation Loss: 0.2272, Validation Accuracy: 0.9308
For epoch:  5
Epoch 6/10, Train Loss: 0.1980, Train Accuracy: 0.9395
Epoch 6/10, Validation Loss: 0.2059, Validation Accuracy: 0.9366
For epoch:  6
Epoch 7/10, Train Loss: 0.1790, Train Accuracy: 0.9451
Epoch 7/10, Validation Loss: 0.1974, Validation Accuracy: 0.9395
For epoch:  7
Epoch 8/10, Train Loss: 0.1614, Train Accuracy: 